In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def generate_dataset(num_examples):
    """Generate a dataset of Tic-Tac-Toe boards and labels"""
    X = np.zeros((num_examples, 9))
    y = np.zeros((num_examples, 9))
    for i in range(num_examples):
        # Generate a random Tic-Tac-Toe board
        board = np.random.randint(0, 3, size=(3, 3))
        X[i] = board.flatten()
        # Calculate the label for the board
        if np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3:
            # player 1 wins
            y[i] = np.array([1 if cell == 1 else 0 for cell in board.flatten()])
        elif np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3:
            # player 2 wins
            y[i] = np.array([1 if cell == 2 else 0 for cell in board.flatten()])
        elif np.any(board == 0):
            # game is still in progress
            y[i] = np.array([0 for cell in board.flatten()])
        else:
            # draw
            y[i] = np.array([0.5 for cell in board.flatten()])
    return X, y

def print_board(board):
  """Print the Tic-Tac-Toe board"""
  print("  0 1 2")
  for i in range(3):
    print(i, end=" ")
    row = " ".join(["X" if cell == 1 else "O" if cell == 2 else " " for cell in board[i]])
    print(row)
   
def get_move(board, player, model):
    """Get the next move for the specified player using the model"""
    if player == 1:
        # Human player's turn
        try:
            row = int(input("Enter row index: "))
            col = int(input("Enter column index: "))
            if row >= 0 and row < 3 and col >= 0 and col < 3 and board[row][col] == 0:
                return row, col
            else:
                print("Invalid move, try again.")
        except ValueError:
            print("Invalid input, try again.")
    else:
        # AI player's turn
        while True:
            # Use the model to predict the best move
            predictions = model.predict(np.array([board.flatten()]))
            move = np.argmax(predictions)
            row = move // 3
            col = move % 3
            if board[row][col] == 0:
                return row, col
            # Add a break statement to exit the loop if a valid move is not found
            break
    # Return a default move if the human player entered invalid input or if the AI player is stuck in an infinite loop
    return 0, 0


def play_game(model):
    """Play a game of Tic-Tac-Toe"""
    board = np.zeros((3, 3))
    current_player = 1
    move_count = 0
    while True:
        print_board(board)
        row, col = get_move(board, current_player, model)
        board[row][col] = current_player
        move_count += 1
        # Switch players after each move
        current_player = 1 if current_player == 2 else 2
        # Check for a win or draw after both players have had a chance to make a move
        if move_count >= 5:
            if (np.any(np.sum(board, axis=0) == 3 * 1) or 
                np.any(np.sum(board, axis=0) == 3 * 2) or
                np.any(np.sum(board, axis=1) == 3 * 1) or 
                np.any(np.sum(board, axis=1) == 3 * 2) or 
                np.sum(np.diag(board)) == 3 * 1 or 
                np.sum(np.diag(board)) == 3 * 2 or 
                np.sum(np.diag(np.fliplr(board))) == 3 * 1 or 
                np.sum(np.diag(np.fliplr(board))) == 3 * 2):
                print("Player {} wins!".format(current_player))
                break
            elif move_count == 9:
                print("It's a draw!")
                break

# Generate a dataset of 1000 Tic-Tac-Toe boards
X, y = generate_dataset(1000)

# Create a model with a single hidden layer with 9 units
model = Sequential()
model.add(Dense(9, input_dim=9, activation='relu'))
model.add(Dense(9, activation='softmax'))

# Compile the model with a custom learning rate and number of epochs
optimizer = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(X, y, batch_size=32, epochs=20, verbose=1)

# Play a game against the AI
play_game(model)


Epoch 1/20
32/32 [==============================] - 1s 4ms/step - loss: 8.0811 - accuracy: 0.1080
Epoch 2/20
32/32 [==============================] - 0s 3ms/step - loss: 9.2417 - accuracy: 0.0800
Epoch 3/20
32/32 [==============================] - 0s 3ms/step - loss: 8.1483 - accuracy: 0.0650
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 7.3101 - accuracy: 0.0560
Epoch 5/20
32/32 [==============================] - 0s 3ms/step - loss: 7.4119 - accuracy: 0.0740
Epoch 6/20
32/32 [==============================] - 0s 4ms/step - loss: 7.4653 - accuracy: 0.1080
Epoch 7/20
32/32 [==============================] - 0s 3ms/step - loss: 7.9041 - accuracy: 0.1070
Epoch 8/20
32/32 [==============================] - 0s 3ms/step - loss: 7.9704 - accuracy: 0.1290
Epoch 9/20
32/32 [==============================] - 0s 3ms/step - loss: 8.0158 - accuracy: 0.1110
Epoch 10/20
32/32 [==============================] - 0s 4ms/step - loss: 8.5740 - accuracy: 0.1120
Epoch 11/20
32/32 [